In [1]:
import spacy
import pandas as pd
from spacy import displacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
import requests, json

nlp = spacy.load('pt')

In [2]:
corpus = pd.read_csv('data/Raw/quinhentas_perguntas.csv', sep=',', header=0)
answers = pd.read_csv('data/Raw/quinhentas_respostas.csv', sep=",", header=0)
corpus['resposta'] = answers['resposta']

In [36]:
url = "https://pt.wikipedia.org/w/api.php?action=raw&format=json&page="

terms = pd.read_csv('data/TreatedTerms/tfidfterms.csv')
for idx, row in terms.iterrows():
    
    myResponse = requests.get(url+row[1])
    #print (myResponse.status_code)
    
    # For successful API call, response code will be 200 (OK)
    if(myResponse.ok):
        jData = json.loads(myResponse.content)
        print(jData)
        print(jData['\*'])
        doc = nlp(jData['wikitext'])
        for token in doc.tokens:
            if terms.contains(token):
                print(token)
        #print("The response contains {0} properties".format(len(jData)))
        #print("\n")
        for key in jData:
            print(key + " : " + str(jData[key]))
    else:
    # If response code is not ok (200), print the resulting http error code with description
        myResponse.raise_for_status()

{'error': {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", '*': 'See https://pt.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}, 'servedby': 'mw1339'}


KeyError: '\\*'

In [12]:
import wikipedia
from spacy.matcher import Matcher
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
wikipedia.set_lang('pt')
terms = pd.read_csv('/home/tiago_duque/Área de Trabalho/Dissertação/textO/data/TreatedTerms/tfidfterms.csv')
termset = set(terms[terms.columns[1]])
matcher = Matcher(nlp.vocab)

for idx, row in terms.iterrows():
    try:
        page = wikipedia.page(row[1])
        #This will move trough the pages linked to this one.
        print(page.categories)
        for token in page.links:
            #tLemma = lemmatizer(token.orth_, token.pos_)[0]
            #print(row[1], tLemma)
            if token.lower() in termset:
                print(row[1], token)
        
    except wikipedia.DisambiguationError as e:
        #Ambiguous - need to try another
        continue
    except wikipedia.PageError as e:
        #No page with ID
        continue

['Categoria:Discografias de músicos do Brasil', 'Categoria:Jullie']
deixar Brasil


KeyboardInterrupt: 